<a href="https://colab.research.google.com/github/ShahistaAfreen/DL_A1/blob/main/DL_A1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import fashion_mnist
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
import wandb
wandb.login() # Login

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [22]:
# Create a new run in a project
run = wandb.init(
    project="Plotting images from each class",
    notes="commit message for the run",
    config={}
)

In [23]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [24]:
np.max(x_train)

255

In [25]:
x_train, x_test = x_train/255.0, x_test/255.0

In [26]:
np.max(x_train)

1.0

In [27]:
x_train.shape

(60000, 28, 28)

In [28]:
type(x_train)

numpy.ndarray

In [29]:
x_train_flattened = x_train.reshape(60000, 28*28)  # Shape: (60000, 784)

In [30]:
x_train_flattened

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
x_train_flattened.shape

(60000, 784)

In [36]:
x_train_with_bias = np.c_[np.ones((60000, 1)), x_train_flattened]

In [37]:
x_train_with_bias.shape

(60000, 785)

Feed Forward Neural Network

In [34]:
def relu(x):
  return np.maximum(0,x)


In [35]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))  # Numerical stability
    return exp_x / np.sum(exp_x, axis=0, keepdims=True)

In [17]:
def FFNN(x_train, weights, hidden_layers):
    activations = [x_train]  # Store activations

    # Forward pass through hidden layers
    for i in range(hidden_layers):
        h_i = np.dot(weights[i], activations[-1])  # Linear transformation
        a_i = relu(h_i)  # Apply ReLU activation
        activations.append(a_i)  # Store activation

    # Output layer (10 neurons for 10 classes)
    h_out = np.dot(weights[-1], activations[-1])
    a_out = softmax(h_out)  # Apply softmax
    activations.append(a_out)

    return activations[-1]

In [18]:
# Network structure
input_size = x_train_with_bias.shape[1]  # 785 = 28x28 +1 images
hidden_layers = 2
hidden_size = 128  # Neurons per hidden layer
output_size = 10  # 10 classes

# Initialize weights and biases
weights = []
w_input = np.random.randn(hidden_size, input_size),  # Input → Hidden 1
weights.append(w_input)


for i in range(hidden_layers):
  w_i = np.random.randn(hidden_size, hidden_size) # hidden(i-1) -> Hidden(i)
  weights.append(w_i)

w_output = np.random.randn(output_size, hidden_size)  # Hidden last → Output
weights.append(w_output)



# Run forward pass
output = FFNN(x_train_with_bias, weights, hidden_layers)
print("Output shape:", output.shape)  # Expected: (10, number of samples)


Output shape: (10, 60000)


In [19]:
output

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [1.00000000e+000, 1.00000000e+000, 1.00000000e+000, ...,
        8.90287972e-196, 0.00000000e+000, 1.12533815e-141],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
       ...,
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000]])

In [20]:
np.max(output)

1.0